In [104]:
import sys
!{sys.executable} -m pip install matplotlib pandas xlrd seaborn tqdm scikit-learn tensorflow openpyxl

  Using cached https://files.pythonhosted.org/packages/bf/58/34bfa8fa17f86333361172b3b502e805195180f19a7496ad0f6149138d55/tensorflow-1.13.1-cp36-cp36m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/0f/39/bdd75b08a6fba41f098b6cb091b9e8c7a80e1b4d679a581a0ccd17b10373/tensorboard-1.13.1-py3-none-any.whl
  Running setup.py bdist_wheel for openpyxl: started
  Running setup.py bdist_wheel for openpyxl: finished with status 'done'
  Stored in directory: C:\Users\shchukin\AppData\Local\pip\Cache\wheels\73\73\c8\bbc0e4a79fc525440783741909e4e47024430f082fc7e0936f
  Running setup.py bdist_wheel for et-xmlfile: started
  Running setup.py bdist_wheel for et-xmlfile: finished with status 'done'
  Stored in directory: C:\Users\shchukin\AppData\Local\pip\Cache\wheels\2a\77\35\0da0965a057698121fc7d8c5a7a9955cdbfb3cc4e2423cad39
Successfully built openpyxl et-xmlfile
  Found existing installation: tensorboard 1.12.2
    Uninstalling tensorboard-1.12.2:
      Successfully uninstalled 

Could not install packages due to an EnvironmentError: [WinError 5] Отказано в доступе: 'c:\\users\\shchukin\\appdata\\local\\programs\\python\\python36\\Lib\\site-packages\\tensorflow\\python\\_pywrap_tensorflow_internal.pyd'
Consider using the `--user` option or check the permissions.

You are using pip version 18.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [36]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import tqdm
import numpy as np

from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, BatchNormalization, LeakyReLU, Activation
from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor

Чтение данных из Excel

In [37]:
sales = pd.read_excel('ДанныеДекабря_2017.xlsx', 'Продажи')
sales = sales.set_index('Material')

ap = pd.read_excel('ДанныеДекабря_2017.xlsx', 'Магазины')
ap = ap.set_index('Material')

stock = pd.read_excel('ДанныеДекабря_2017.xlsx', 'Остатки')
stock = stock.set_index('Material')

price = pd.read_excel('ДанныеДекабря_2017.xlsx', 'Цены')
price = price.set_index('Material')

hierarchy = pd.read_excel('hierarchy.xlsx')

sell = pd.read_excel('Акции.xlsx')

Создание DataFrame для анализа зависимостей между переменными по артикулам

In [86]:
data_df = pd.melt(sales.reset_index(), id_vars=['Material'], value_vars=list(sales.columns), var_name='date', value_name='sales').sort_values(by=['Material','date'], ascending=False)
data_df = pd.merge(data_df, pd.melt(stock.reset_index(), id_vars=['Material'], value_vars=list(stock.columns), var_name='date', value_name='stock_1').sort_values(by=['Material','date'], ascending=False).shift(-1), on=['Material', 'date'])
data_df = pd.merge(data_df, pd.melt(price.reset_index(), id_vars=['Material'], value_vars=list(price.columns), var_name='date', value_name='price_1').sort_values(by=['Material','date'], ascending=False).shift(-1), on=['Material', 'date'])
data_df = pd.merge(data_df, pd.melt(ap.reset_index(), id_vars=['Material'], value_vars=list(ap.columns), var_name='date', value_name='ap_1').sort_values(by=['Material','date'], ascending=False).shift(-1), on=['Material', 'date'])
data_df.loc[:,'sales_1'] = data_df['sales'].shift(-1)
data_df.loc[:,'sales_2'] = data_df['sales_1'].shift(-1)
data_df.loc[:,'sales_12'] = data_df['sales_1'].shift(-11)
data_df.loc[:,'sales_diff_1'] = data_df['sales_1'] - data_df['sales_1'].shift(-1)
data_df.loc[:,'sales_diff_2'] = data_df['sales_1'] - data_df['sales_1'].shift(-2)
data_df.loc[:,'sales_diff_12'] = data_df['sales_1'] - data_df['sales_1'].shift(-11)
data_df.loc[:,'stock_2'] = data_df['stock_1'].shift(-1)
data_df.loc[:,'stock_diff_1'] = data_df['stock_1'] - data_df['sales_1'].shift(-1)
data_df.loc[:,'stock_diff_2'] = data_df['stock_1'] - data_df['sales_1'].shift(-2)
data_df.loc[:,'price_2'] = data_df['price_1'].shift(-1)
data_df.loc[:,'ap_2'] = data_df['ap_1'].shift(-1)

data_df = data_df.set_index('date')
    
data_df.loc[:,'month'] = data_df.index.month

data_df = data_df.dropna()

In [5]:
del sales
del ap
del stock
del price
del hierarchy
del sell

Сохраняем собранные данные в файл для ускорения в будущем

In [4]:
data_df.to_excel('full_train_data.xlsx')

Выгружаем данные для обучения из файла

In [6]:
data_df = pd.read_excel('full_train_data.xlsx')

Создаем DF корелляции

In [87]:
corr_df = data_df.groupby('Material').corr().abs().sales.fillna(0).unstack(level=0).T

Выделение групп артикулов для ращного прогнозирования на основе коррелляции

In [94]:
#Cluster the data
kmeans = KMeans(n_clusters=5, random_state=0).fit(corr_df)
labels = kmeans.labels_

#Glue back to originaal data
corr_df['clusters'] = labels

# Создание нормализованной DF
corr_overal_df = corr_df.groupby('clusters').mean()

corr_overal_norm_df = (corr_overal_df-corr_overal_df.min())/(corr_overal_df.max()-corr_overal_df.min())

In [106]:
corr_df.to_excel('material_clusters.xlsx')

In [95]:
corr_overal_df.style.background_gradient(cmap='viridis')

,sales,stock_1,price_1,ap_1,sales_1,sales_2,sales_12,sales_diff_1,sales_diff_2,sales_diff_12,stock_2,stock_diff_1,stock_diff_2,price_2,ap_2,month
clusters,,,,,,,,,,,,,,,,
0,1,0.629716,0.403884,0.658637,0.357855,0.245474,0.200724,0.133925,0.15045,0.276296,0.540344,0.611605,0.610299,0.30461,0.579115,0.237505
1,1,0.925932,0.788833,0.917534,0.740039,0.530359,0.149284,0.321507,0.403406,0.616876,0.758346,0.901525,0.905526,0.67885,0.807063,0.181179
2,1,0.792636,0.555646,0.796083,0.599668,0.490098,0.216058,0.143463,0.18204,0.442609,0.701932,0.769004,0.768968,0.461346,0.730991,0.19326
3,1,0.868587,0.791363,0.856805,0.242984,0.114517,0.132398,0.138204,0.152922,0.241772,0.392618,0.796508,0.785888,0.52586,0.474367,0.194701
4,0.998884,0.373051,0.263509,0.442281,0.299987,0.164802,0.185248,0.16093,0.17757,0.215024,0.30092,0.352688,0.355843,0.183728,0.367165,0.237097


Вычисление колонок у кластера для использования для прогнозирования

In [96]:
# Будем отсекать по средней корелляции, за исключением артикулов, у которых нет данных
threshold = corr_overal_df[corr_overal_df.sales > 0.1].mean()
# Инициализация копии DF
corr_thresh_df = corr_overal_df.copy()
# Прогон сравнения по всем колонкам
for col in corr_overal_df.columns:
    corr_thresh_df[col] = (corr_overal_df[col] >= threshold[col]).astype(int)
    
corr_thresh_df.style.background_gradient(cmap='viridis')

,sales,stock_1,price_1,ap_1,sales_1,sales_2,sales_12,sales_diff_1,sales_diff_2,sales_diff_12,stock_2,stock_diff_1,stock_diff_2,price_2,ap_2,month
clusters,,,,,,,,,,,,,,,,
0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1
1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,0
2,1,1,0,1,1,1,1,0,0,1,1,1,1,1,1,0
3,1,1,1,1,0,0,0,0,0,0,0,1,1,1,0,0
4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1


Создаем DF для всех материалов для обучения нейронки

In [97]:
train_data = data_df.merge(corr_df.clusters.to_frame(), how='left', left_on='Material', right_index=True)

Создаем тренировочные данные

In [98]:
X = train_data[[col for col in train_data.columns if col not in ['sales', 'Material']]]

Y = train_data.sales.to_frame()

scaler = MinMaxScaler()

print(scaler.fit(X))
print(scaler.fit(Y))

xscale=scaler.transform(X)
yscale=scaler.transform(Y)

X_train, X_test, y_train, y_test = train_test_split(xscale, yscale)

c:\users\shchukin\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


MinMaxScaler(copy=True, feature_range=(0, 1))


c:\users\shchukin\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


MinMaxScaler(copy=True, feature_range=(0, 1))


In [109]:
# 82.6%
model = Sequential()
model.add(Dense(32, input_dim=16, kernel_initializer='normal'))
model.add(Dense(16))
model.add(Dense(1))
model.summary()

model.compile(loss='mse', optimizer='adam', metrics=['mse','mae'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 32)                544       
_________________________________________________________________
dense_20 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 17        
Total params: 1,089
Trainable params: 1,089
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(X_train, y_train, epochs=200, batch_size=50, validation_data=(X_test, y_test), shuffle=True, verbose=0)

In [ ]:
plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='best')
plt.show()

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
prediction = model.predict(X_test)

accuracy = 1 - sum(abs(scaler.inverse_transform(prediction) - scaler.inverse_transform(y_test))) / sum(scaler.inverse_transform(y_test))

print('Total accuracy = %.2f%%' % (accuracy * 100))

In [33]:
# Сохраним модель полностью в единый HDF5 файл

In [34]:
model.save('best_model.h5')